In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

def split_data(data_dir, test_size=0.3, val_size=0.5, random_state=42):
    # Loading the data from .npy files
    image_data = np.load(os.path.join(data_dir, "image_data.npy"))
    labels = np.load(os.path.join(data_dir, "labels.npy"))

    # Spliting the data into training, testing, and validation sets
    X_train, X_temp, y_train, y_temp = train_test_split(image_data, labels, test_size=test_size, random_state=random_state)
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=val_size, random_state=random_state)

    return X_train, y_train, X_test, y_test, X_val, y_val

data_dir = r"F:\End_To_End_project\Kidney_Disease_Classification_DL\data\preprocessed_data"
X_train, y_train, X_test, y_test, X_val, y_val = split_data(data_dir, test_size=0.3, val_size=0.5)


In [2]:
print(f"Shape of x_train data :{X_train.shape}")
print(f"Shape of y_train data :{y_train.shape}")
print(f"Shape of x_val data :{X_val.shape}")
print(f"Shape of y_val data :{y_val.shape}")
print(f"Shape of X_test data :{X_test.shape}")
print(f"Shape of y_test data :{y_test.shape}")

Shape of x_train data :(3855, 224, 224)
Shape of y_train data :(3855,)
Shape of x_val data :(827, 224, 224)
Shape of y_val data :(827,)
Shape of X_test data :(826, 224, 224)
Shape of y_test data :(826,)


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the CNN model
model = models.Sequential([
    # Convolutional layers
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    # Flatten the feature maps
    layers.Flatten(),
    # Fully connected layers
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5), 
    layers.Dense(4, activation='softmax') 
])


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [5]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


early_stopping = EarlyStopping(
    monitor='val_loss',        # it will Monitor validation loss
    patience=10,               # Number of epochs with no improvement before stopping
    restore_best_weights=True  # Restore the model weights to the best epoch
)

checkpoint = ModelCheckpoint(
    'best_model.h5',  # File to save the best model
    monitor='val_accuracy',  # Monitor validation accuracy
    save_best_only=True,     # Save only the best model
    mode='max',             # Save the model when accuracy is maximized
    verbose=1
)

history = model.fit(
    X_train,
    y_train,
    epochs=10,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, checkpoint]
)


Epoch 1/10
121/121 [==============================] - ETA: 0s - loss: 0.5962 - accuracy: 0.7512
Epoch 1: val_accuracy improved from -inf to 0.99154, saving model to best_model.h5


f:\End_To_End_project\Kidney_Disease_Classification_DL\kidney\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


121/121 [==============================] - 366s 3s/step - loss: 0.5962 - accuracy: 0.7512 - val_loss: 0.0649 - val_accuracy: 0.9915
Epoch 2/10
121/121 [==============================] - ETA: 0s - loss: 0.0896 - accuracy: 0.9696
Epoch 2: val_accuracy improved from 0.99154 to 0.99274, saving model to best_model.h5
121/121 [==============================] - 362s 3s/step - loss: 0.0896 - accuracy: 0.9696 - val_loss: 0.0234 - val_accuracy: 0.9927
Epoch 3/10
121/121 [==============================] - ETA: 0s - loss: 0.0362 - accuracy: 0.9891
Epoch 3: val_accuracy improved from 0.99274 to 0.99516, saving model to best_model.h5
121/121 [==============================] - 407s 3s/step - loss: 0.0362 - accuracy: 0.9891 - val_loss: 0.0150 - val_accuracy: 0.9952
Epoch 4/10
121/121 [==============================] - ETA: 0s - loss: 0.0190 - accuracy: 0.9943
Epoch 4: val_accuracy improved from 0.99516 to 0.99758, saving model to best_model.h5
121/121 [==============================] - 487s 4s/step - 